In [227]:
from nbconvert.preprocessors import ExecutePreprocessor
import pandas as pd
import nbformat
import numpy as np

In [ ]:

from nbclient import NotebookClient
import pandas as pd

# Define paths and initial parameters
notebook_path = 'MixedLogit.ipynb'#set to other name if testing other DGP
output_csv = 'output.csv'
param_name = 'NUM_USER'
param_values =[500,1000,2000,10000,20000]
# param_name = 'NUM_Product'  
# param_values =[4,10,20,50,100]
# param_name = 'treatment_percentage'
# param_values =[0.2,0.5,0.8]



# Function to run the notebook with an updated parameter
def run_notebook(param_value):
    # Load the notebook
    with open(notebook_path) as f:
        notebook = nbformat.read(f, as_version=4)

    cell =  notebook['cells'][2] #set to 2 if numb pf  user, 3 if number of product, and 4 if treatment percentage
    cell['source'] = f"{param_name} = {param_value}"  # Set the parameter
    
    # Execute the notebook
    client = NotebookClient(notebook)
    # Get the output from the final cell

    try:
        client.execute()
    except Exception as e:
        print(f"Error during execution for {param_name} = {param_value}: {e}")
        return None  # Return None if there's an error
    final_cell_output = notebook['cells'][-1]['outputs']
    # Extract and parse the text output of the final cell
    result_text = None
    for output in final_cell_output:
        if output.get('output_type') == 'stream' and 'text' in output:
            result_text = output['text']
            break
    
    # Parse the result text into a list of floats
    if result_text:
        result_values = list(map(float, result_text.strip().split()))
        return [param_value] + result_values
    return None

# Run the notebook 200 times with different parameter values and collect results
results = []
for param in param_values:
    for i in range(200):
        print(f"Run {i+1} for {param_name} = {param}")
        result = run_notebook(param)
        if result:
            results.append(result)
# Convert results to a DataFrame and save to CSV
columns = ['Parameter'] + [f'Value_{i+1}' for i in range(len(results[0]) - 1)]
df = pd.DataFrame(results, columns=columns)
df.to_csv(output_csv, index=False)

print(f"All runs completed. Results saved to {output_csv}")
